softmax 예제

In [21]:
import numpy as np
a=np.array([0.3,2.9,4.0])

exp_a = np.exp(a)
print(exp_a)

sum_exp_a = np.sum(exp_a)
print(sum_exp_a)

y=exp_a / sum_exp_a
print(y)

[ 1.34985881 18.17414537 54.59815003]
74.1221542101633
[0.01821127 0.24519181 0.73659691]


softmax 함수

In [24]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y=exp_a / sum_exp_a

    return y

softmax : 지수함수 너무 커서 overflow 발생  
그래서 입력 신호 중 최댓값을 사용하여 값 조정  
softmax의 지수 함수를 계산할 때 어떤 정수를 더해도 결과 같음!!

In [27]:
a=np.array([1010,1000,990])
np.exp(a)/np.sum(np.exp(a))

c=np.max(a)
print(a-c)

np.exp(a-c) / np.sum(np.exp(a-c))

[  0 -10 -20]


C:\Users\배동혁\AppData\Local\Temp\ipykernel_12816\1600684083.py:2: RuntimeWarning: overflow encountered in exp
  np.exp(a)/np.sum(np.exp(a))
C:\Users\배동혁\AppData\Local\Temp\ipykernel_12816\1600684083.py:2: RuntimeWarning: invalid value encountered in divide
  np.exp(a)/np.sum(np.exp(a))


array([9.99954600e-01, 4.53978686e-05, 2.06106005e-09])

조정된 softmax 함수

In [30]:
def softmax(a):
    c=np.max(a)
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y

In [32]:
a=np.array([0.3,2.9,4.0])
y=softmax(a)
print(y)
np.sum(y)

[0.01821127 0.24519181 0.73659691]


1.0

In [46]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!
(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [54]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

def img_show(img):
   pil_img=Image.fromarray(np.uint8(img))
   pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img=x_train[0]
label=t_train[0]
print(label)

print(img.shape)
img=img.reshape(28,28)
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


In [74]:
def get_data():
    (x_train,t_train), (x_test, t_test)=load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network=pickle.load(f)

    return network

def predict(network, x):
    w1, w2, w3=network['w1'], network['w2'], network['w3']
    b1, b2, b3=network['b1'], network['b2'], network['b3']

    a1=np.dot(x,w1) + b1
    z1=sigmoid(a1)
    a2=np.dot(z1,w2) + b2
    z2=sigmoid(a2)
    a3=np.dot(z2,w3) + b3
    y=softmax(a3)

    return y

In [76]:
x, t = get_data()
network= init_netowrk()

accuracy_cnt=0
for i in range(len(x)):
    y=predict(network, x[i])
    p=np.argmax(y)
    if p==t[i]:
        accuracy_cnt+=1

print("Accuracy:" + str(float(accuracy_cnt)/len(x)))

NameError: name 'init_netowrk' is not defined

In [78]:
x, _ = get_data()
network=init_network()
w1, w2, w3 = network['w1'], network['w2'], network['w3']

x.shape
x[0].shape
w1.shape
w2.shape
w3.shape

NameError: name 'pickle' is not defined

In [84]:
x, t= get_data()
network=init_network()

batch_size=100
accuracy_cnt=0

for i in range(0, len(x), batch_size):
    x_batch=x[i:i+batch_size]
    y_natch=predict(network, x_batch)
    p=np.argmax(y_batch, axis=1)
    accuracy_cnt+=np.sum(p==t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt/len(x))))

NameError: name 'pickle' is not defined

In [15]:
list(range(0, 10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [86]:
list(range(0,10,3))

[0, 3, 6, 9]

In [88]:
x=np.array([[0.1,0.8,0.1],[0.3,0.1,0.6],[0.2,0.5,0.3],[0.8,0.1,0.1]])
y=np.argmax(x,axis=1)
print(y)

[1 2 1 0]


In [90]:
y=np.array([1,2,1,0])
t=np.array([1,2,0,0])
print(y==t)
np.sum(y==t)

[ True  True False  True]


3